
# Note book for the project 2, part 2




We begin by importing usefull libraries and files.

In [ ]:
import numpy as np
import pandas as pd
from split_data import split_data
%matplotlib inline
#!/usr/bin/env python3
from scipy.sparse import *
from sklearn import linear_model, preprocessing, neural_network
import numpy as np
from IOTweets import *
from ProcessTweets import *
import pickle
import random
#from joblib import Parallel, delayed
import csv
from feature_helper import *
from pickle_vocab import *
from cooc import *
from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer(False)

# 0.Variables
Then we define the name of the files we will sometimes use as well as the number of dimensions

In [ ]:
#Define filenames and variables

#embeddings

embeddings = "embeddings_full.npy"

embeddings_ts =      'embeddings_ts.npy'
embeddings_te =      'embeddings_te.npy'
embeddings_ts_full = 'embeddings_ts_full.npy'
embeddings_te_full = 'embeddings_te_full.npy'

embeddings_preprocessed = 'embeddings_preprocessed.npy'
#tweets
pos_ts_tweets =      'cleaned_pos_bitri=True'
neg_ts_tweets =      'cleaned_neg_bitri=True'
pos_ts_full_tweets = 'cleaned_pos_full_bitri=True'
neg_ts_full_tweets = 'cleaned_neg_full_bitri=True'
te_full_tweets =     'cleaned_test_bitri=True'

#vocab
file_vocab = 'vocab.pkl'
file_vocab_preprocess = 'vocab_cut_preprocessed_full.pkl'


#coocurrence matrices
cooc_full = 'cooc_full.pkl'
cooc_partial = 'cooc.pkl'
cooc_preprocessed = 'cooc_preprocessed.pkl'
cooc_stemed_full = "cooc_stemed_full.pkl"

#Features variables
#pertinence = see construct_features.py
nb_dim = 20

# Helper Functions
Here we define our own GloVe function and an accuracy detector.

In [ ]:
def accuracy(prediction, actual_emotions):
    return (1 - (np.sum(np.abs(actual_emotions-prediction))/(2*len(actual_emotions)))) * 100


def GloVe(file_name="cooc_partial", destination=embeddings_ts, embedding_dim = 50):
    #load coocurence matrix
    with open(file_name, 'rb') as f:
        cooc = pickle.load(f)    
    
    nmax = 100
    
    eta = 0.001
    alpha = 3 / 4
    epochs = 10
    
    xs = np.random.normal(size=(cooc.shape[0], embedding_dim))
    ys = np.random.normal(size=(cooc.shape[1], embedding_dim))
   
    #Construct vector representations xs for words
    for epoch in range(epochs):
        print("epoch {}".format(epoch))
        loading_counter = 0
        for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
            
            f = ((n / nmax)**alpha) if n < nmax else 1
            inter_cost = (xs[ix]@(ys[jy]) - np.log(n))
            # We compute the gradients for both context and main vector words
            grad_main = f * inter_cost * ys[jy]
            grad_context = f * inter_cost * xs[ix]
    
            # Update the vector words
            xs[ix] = xs[ix] - (eta * grad_main)
            ys[jy] = ys[jy] - (eta * grad_context)
            
            if loading_counter%20000==1:
                    print("{:.1f}".format(loading_counter/len(cooc.col)*100), "%", end='\r')
            loading_counter+=1
            
    #Store xs in destination file
    np.save(file=destination, arr=xs)
    print("finished")

# Creating coocurence matrix


In [ ]:
# Build the global vocab, which contains: pos, neg and the test vocab
build_global_vocab(False, True, 5)

# Create the vocab from the global vocab
to_vocab_pickle("global_vocab_cut=5")

# Create the cooc matrix from the
to_cooc(path_vocab="global_vocab.pkl", path_tweets_pos="cleaned_vocab_pos_bitri=True", path_tweets_neg="cleaned_vocab_neg_bitri=True")

# create all tweets
        
#commande shell to happen all tweets
#!cat cleaned_neg_bitri\=True cleaned_pos_bitri\=True cleaned_test_bitri\=True > all_tweets_cleaned 

Now given the co-occurrence matrix and the vocabulary, it is not hard to train GloVe word embeddings, that is to compute an embedding vector for wach word in the vocabulary. We suggest to implement SGD updates to train the matrix factorization.

Construct Features for the Training Texts: Load the training tweets and the built GloVe word embeddings. Using the word embeddings, construct a feature representation of each training tweet (by averaging the word vectors over all words of the tweet).

In [ ]:
GloVe(file_name="cooc_bitri=True.pkl", destination="embeddings_bitri=True")

# Classify tweets
Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels indicate if a tweet used to contain a 🙂 or 🙁 smiley.

In [ ]:
def build_and_predict(pertinent_lb):    
    
    
    #define relevant_vocab file to use
    relevant_vocab = 'relevant_vocab_pert=0.3_count=300'#'relevant_vocab_lb='+str(pertinent_lb)+'.txt'
    
    #load ratios into a dictionary
    weights = extract_relevant(relevant_vocab)
    
    #Split positive tweets into training and testing sets
    pos_tweets = np.array(open(pos_ts_full_tweets, 'r').readlines()) 
    labels_pos = np.ones(len(pos_tweets))
    pos_tr, labels_pos_tr, pos_te, labels_pos_te = split_data(pos_tweets, labels_pos, 0.996)
    
   

    #Split negative tweets into training and testing sets
    neg_tweets = np.array(open(neg_ts_full_tweets, 'r').readlines())
    labels_neg = np.full(len(neg_tweets), -1)
    neg_tr, label_neg_tr, neg_te, labels_neg_te = split_data(neg_tweets, labels_neg, 0.996)
            
    #Find features for each tweet with at least one word within vocab, get indices of unpredictable tweets
    #for pos tweetspos_tr
    pos_tr_feat, invalid_pos_tr = construct_features("global_vocab_cut=5", pos_tr, "embeddings_bitri=True.npy", "relevant_vocab_pert=0.3_count=300", -1)
    pos_te_feat, invalid_pos_te = construct_features("global_vocab_cut=5", pos_te, "embeddings_bitri=True.npy", "relevant_vocab_pert=0.3_count=300", -1)

    #for neg tweets
    neg_tr_feat, invalid_neg_tr = construct_features("global_vocab_cut=5", neg_tr, "embeddings_bitri=True.npy", "relevant_vocab_pert=0.3_count=300", -1)
    neg_te_feat, invalid_neg_te = construct_features("global_vocab_cut=5", neg_te, "embeddings_bitri=True.npy", "relevant_vocab_pert=0.3_count=300", -1)
    
    print(len(pos_tr_feat), len(invalid_pos_tr))
    print(len(pos_te_feat), len(invalid_pos_te))
    print(len(neg_tr_feat), len(invalid_neg_tr))
    print(len(neg_te_feat), len(invalid_neg_te))
    
    #Initialize classifier and scaler
    neural = neural_network.MLPClassifier(hidden_layer_sizes=(nb_dim, nb_dim, nb_dim/2, 2))
    scaler = preprocessing.StandardScaler()

    #fit classifier on predictable tweets
    X = np.concatenate((pos_tr_feat, neg_tr_feat))
    Y = np.concatenate((np.ones(len(pos_tr_feat)), np.full(len(neg_tr_feat), -1)))
    X = scaler.fit_transform(X, Y)
    neural = neural.fit(X, Y)

    #scale data that should be predicted
    pos_te_feat_scaled = pos_te_feat
    pos_te_feat_scaled = scaler.fit_transform(pos_te_feat_scaled, np.ones(len(pos_te_feat))) 

    neg_te_feat_scaled = neg_te_feat
    neg_te_feat_scaled = scaler.fit_transform(neg_te_feat_scaled, np.ones(len(neg_te_feat))) 

    #predict predictable tweets
    pos_prediction = neural.predict(pos_te_feat_scaled)
    neg_prediction = neural.predict(neg_te_feat_scaled)

    #merge with unpredictable tweets predictions
    pos_labels = assemble(pos_prediction, invalid_pos_te)
    neg_labels = assemble(neg_prediction, invalid_neg_te)

    #merge all predictions
    labels = np.concatenate((pos_labels, neg_labels))
    true_labels = np.concatenate((np.ones(len(pos_labels)), np.full(len(neg_labels), -1)))

    return labels, true_labels    
    

In [ ]:
labels, true_labels = build_and_predict(0.3)

# submission procedure
Prediction: Predict labels for all tweets in the test set.

Submission / Evaluation: Submit your predictions to kaggle, and verify the obtained misclassification error score. (You can also use a local separate validation set to get faster feedback on the accuracy of your system). Try to tune your system for best evaluation score.

In [ ]:
def submit(pertinence):
    #Load words from tweet set
    #xs = np.load(embeddings_ts_full)
    
    #define relevant_vocab file to use
    relevant_vocab = 'relevant_vocab_full_lb=1000.txt'
    
    #load ratios into a dictionary
    weights = extract_relevant(relevant_vocab)
    
    print(len(new_vocab))
    
    pos_tweets = np.array(open(file=pos_ts_full_tweets, mode='r', encoding="utf8").readlines()) 
    neg_tweets = np.array(open(file=neg_ts_full_tweets, mode='r', encoding="utf8").readlines()) 
    te_tweets = np.array(open(file=te_full_tweets, mode='r', encoding="utf8").readlines()) 

    
    #Find features for each tweet with at least one word within vocab, get indices of unpredictable tweets
    #for training tweets
    pos_ts_full_feat, invalid_pos_ts_full = construct_features("global_vocab_cut=5", pos_tweets, "embeddings_bitri=True.npy", "relevant_vocab_pert=0.3_count=300", 5)

    neg_ts_full_feat, invalid_neg_ts_full = construct_features("global_vocab_cut=5", neg_tweets, "embeddings_bitri=True.npy", "relevant_vocab_pert=0.3_count=300", 5)
    
    print(len(pos_ts_full_feat), len(invalid_pos_ts_full) )
    print(len(neg_ts_full_feat), len(invalid_neg_ts_full) )
    
    #for test tweets
    te_full_feat, invalid_te_full = construct_features("global_vocab_cut=5", te_tweets, "embeddings_bitri=True.npy", "relevant_vocab_pert=0.3_count=300", 5)
    print(len(te_full_feat), len(invalid_te_full) )
    
     #Initialize classifier and scaler
    neural = neural_network.MLPClassifier(hidden_layer_sizes=(nb_dim, nb_dim, nb_dim/2, 2))
    scaler = preprocessing.StandardScaler()

    #fit classifier on predictable tweets
    X = np.concatenate((pos_ts_full_feat, neg_ts_full_feat))
    Y = np.concatenate((np.ones(len(pos_ts_full_feat)), np.full(len(neg_ts_full_feat), -1)))
    X = scaler.fit_transform(X, Y)
    neural = neural.fit(X, Y)

    #scale data that should be predicted
    te_full_feat_scaled = te_full_feat
    te_full_feat_scaled = scaler.fit_transform(te_full_feat_scaled, np.ones(len(te_full_feat))) 


    #predict predictable tweets
    te_prediction = neural.predict(te_full_feat_scaled)

    #merge with unpredictable tweets predictions
    labels = assemble(te_prediction, invalid_te_full)
   
    return labels
    
    
    

In [ ]:
labels = submit(35)

In [ ]:
labels

In [ ]:
predict_te = labels
with open('submission.csv', 'w',) as f2:
    fields = ('Id', 'Prediction')
    wr = csv.DictWriter(f2, fieldnames=fields, lineterminator = '\n')
    wr.writeheader()
    
    for id_tweet, prediction in enumerate(predict_te):
        wr.writerow({'Id':id_tweet+1, 'Prediction': (int)(prediction)})
        print(id_tweet)

# create submission file

In [ ]:
predict_te = submit(35)
with open('submission.csv', 'w',) as f2:
    fields = ('Id', 'Prediction')
    wr = csv.DictWriter(f2, fieldnames=fields, lineterminator = '\n')
    wr.writeheader()
    
    for id_tweet, prediction in enumerate(predict_te):
        wr.writerow({'Id':id_tweet+1, 'Prediction': (int)(prediction)})
        print(id_tweet)